# Billings

Zoning Data Layer: https://billingsgis.com/arcgis_public/rest/services/ArcOnline_Public/COB_Zoning/MapServer/4

Note: a bit of manual clean up is required to remove the parcels near Laurel

This is intended to be run in the middle of data_prep.ipynb. If running standalone, run the first two cells of `data_prep.ipynb` first.

In [37]:
city = cities[index_of('Billings')]


In [38]:
city

{'name': 'Billings', 'zoning_field': 'ZONE_CODE', 'county': 'Yellowstone', 'residential_zones': ['N1', 'N2', 'N3', 'NX1', 'NX2', 'NX3', 'RMH'], 'gdb': 'cities\\Billings.gdb', 'layer': 'cities\\Billings.gdb\\Parcels_Billings'}

In [31]:
# units allowed by underlying zoning
codeblock = """
def eval_zoning(zone):
    if zone in ['N3', 'RMH']:
        return 1
    elif zone in ['N1', 'N2']:
        return 2
    elif zone in ['NX1']:
        return 4
    elif zone in ['NX2', 'NX3']:
        return 5
    else:
        return 0
"""

arcpy.management.CalculateField(
    city['layer'], 
    "zoning_allows", 
    "eval_zoning(!zoning!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'cities\\Billings.gdb\\Parcels_Billings'>

In [32]:
codeblock = """
def eval_zoning(zone, lot_size):
    return 5
"""

arcpy.management.CalculateField(
    city['layer'], 
    "lot_size_allows", 
    "eval_zoning(!zoning!, !lot_size!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'cities\\Billings.gdb\\Parcels_Billings'>

In [39]:
codeblock = """
def eval_zoning(zone, lot_width):
    if zone == 'N1' and lot_width >= 20:
        return 2
    elif zone == 'N2':
        if lot_width >= 50:
            return 2
        else:
            return 0
    elif zone == 'N3':
        if lot_width >= 65:
            return 1
        else:
            return 0
    elif zone == 'NX1':
        if lot_width >= 50:
            return 4
        elif lot_width >= 20:
            return 2
        else:
            return 0
    elif zone == 'NX2':
        if lot_width >= 50:
            return 5
        elif lot_width >= 20:
            return 2
        else:
            return 0
    elif zone == 'NX3':
        if lot_width >= 50:
            return 5
        else:
            return 0
    else:
        return 0
"""

arcpy.management.CalculateField(
    city['layer'], 
    "lot_width_allows", 
    "eval_zoning(!zoning!, !lot_width!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'cities\\Billings.gdb\\Parcels_Billings'>

In [40]:
codeblock = """
def categorize(zone):
    if zone in ['N1', 'N2', 'NX1', 'NX2', 'NX3']:
        return "Multifamily"
    elif zone in ['N3', 'RMH']:
        return "Single-Family"
    else:
        return "Unknown"
"""
arcpy.management.CalculateField(
    city['layer'], 
    "category", 
    "categorize(!zoning!)",
    "PYTHON3",
    codeblock,
    "TEXT"
)

<Result 'cities\\Billings.gdb\\Parcels_Billings'>